<a href="https://colab.research.google.com/github/merriekay/CS167Code/blob/main/Day15Notes_dummies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 15 Notes
- Dummmy Variables
- Dummy Regressor

## **Project \#1 Tips**: Dummy Regressor and get_dummies
This is relevant to **Project #1** part 3 (**Data Preparation**) and Part 5 (**Baseline Performance**). 
- [pandas.get_dummies](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html) provides a way to transform categorical predictor columns into dummy variables. I will show you an example of how to use this function. 
- [sklearn.dummy.DummyRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyRegressor.html) provides an easy way to test how a niave machine learning strategy (guessing randomly, or guessing the mean), would perform on your data. The idea is to use this as a baseline, and then be able to beat this baseline using the other models in your project.

Let's start with the Vehicles dataset that we've used in previous notebooks:


In [41]:
import pandas
import numpy
from google.colab import drive
drive.mount('/content/drive')
vehicles = pandas.read_csv('/content/drive/MyDrive/CS167_S21/CS167_datasets/vehicles.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (70,71,72,73,74,76,79) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [42]:
data = vehicles[['drive', 'fuelType', 'cylinders', 'displ', 'comb08']]
data.head()

,drive,fuelType,cylinders,displ,comb08
0,Rear-Wheel Drive,Regular,4.0,2.0,21
1,Rear-Wheel Drive,Regular,12.0,4.9,11
2,Front-Wheel Drive,Regular,4.0,2.2,27
3,Rear-Wheel Drive,Regular,8.0,5.2,11
4,4-Wheel or All-Wheel Drive,Premium,4.0,2.2,19


## Create Dummy Variables
Let's create dummy variables for the 'drive' column and the 'fuelType' column. If you want to check what the unique values within these columns are (i.e. the number of columns that will be added to your dataset), use `data.drive.unique()`, and `data.fuelType.unique()`

In [53]:
print('Values for the drive column: '+ str(data.drive.unique()))
print('Values for the fuelType column: '+ str(data.fuelType.unique()))
print('**********************')
print('This should add {} columns to the dataset, but subtract 2.'.format((len(data.drive.unique())+len(data.fuelType.unique()))))
print('Original shape: {}'.format(data.shape))

Values for the drive column: ['Rear-Wheel Drive' 'Front-Wheel Drive' '4-Wheel or All-Wheel Drive' nan
 '2-Wheel Drive' 'All-Wheel Drive' '4-Wheel Drive'
 'Part-time 4-Wheel Drive']
Values for the fuelType column: ['Regular' 'Premium' 'Diesel' 'CNG' 'Gasoline or natural gas'
 'Gasoline or E85' 'Electricity' 'Gasoline or propane' 'Premium or E85'
 'Midgrade' 'Premium Gas or Electricity' 'Regular Gas and Electricity'
 'Premium and Electricity' 'Regular Gas or Electricity']
**********************
This should add 22 columns to the dataset, but subtract 2.
Original shape: (42103, 5)


In [52]:
#note that you can't call this function more than once on the same column since it drops the column. 
vehicles_dummies = pandas.get_dummies(data, columns = ['drive', 'fuelType'])
print('Dummy Dataset shape: {}'.format(vehicles_dummies.shape))
vehicles_dummies.head()

Dummy Dataset shape: (42103, 24)


,cylinders,displ,comb08,drive_2-Wheel Drive,drive_4-Wheel Drive,drive_4-Wheel or All-Wheel Drive,drive_All-Wheel Drive,drive_Front-Wheel Drive,drive_Part-time 4-Wheel Drive,drive_Rear-Wheel Drive,fuelType_CNG,fuelType_Diesel,fuelType_Electricity,fuelType_Gasoline or E85,fuelType_Gasoline or natural gas,fuelType_Gasoline or propane,fuelType_Midgrade,fuelType_Premium,fuelType_Premium Gas or Electricity,fuelType_Premium and Electricity,fuelType_Premium or E85,fuelType_Regular,fuelType_Regular Gas and Electricity,fuelType_Regular Gas or Electricity
0,4.0,2.0,21,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,12.0,4.9,11,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,4.0,2.2,27,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,8.0,5.2,11,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,4.0,2.2,19,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


## Build a Dummy Regressor that predicts the average mpg
Now, let's use [sklearn.dummy.DummyRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyRegressor.html) to build a dummyregressor that naively chooses the mean of the target column as it's prediction. This should provide a baseline performance that we should be able to beat using other models. 

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn import metrics

predictors = data.columns.drop('comb08')
target = 'comb08'

train_data, test_data, train_sln, test_sln = train_test_split(data[predictors], data[target], test_size = 0.2, random_state=41)

dummy = DummyRegressor()
dummy.fit(train_data, train_sln)
naive_preds = dummy.predict(test_data)
print('MSE: '+ str(metrics.mean_squared_error(naive_preds, test_sln)))
print('MAE: '+ str(metrics.mean_absolute_error(naive_preds, test_sln)))
print('R2: '+ str(metrics.r2_score(naive_preds, test_sln)))

MSE: 68.96246588129243
MAE: 4.522588314756327
R2: -5.463769454227446e+30


That means that other models should be able to perform better than the metrics that just printed. If we were to get an MSE below 69, an MAE below 4.5 or an R2 score below zero, we should know that that's probably not a winning model. 